## Basics of Mobile Robotics

## Table of Contents

* [Introduction](#chapter1)
    * [Framework](#section_1_1)
    * [Modules](#section_1_2)
    * [Environment](#section_1_3)
* [Computer Vision](#chapter2)
    * [Vision initialization and calibration](#section_2_1)
    * [Prepocessing](#section_2_1)
    * [Map initialization](Section_2_2)
        * [Obstacles Detection](#section_2_2_1)
        * [Goal Detection](#section_2_2_2)
        * [Thymio Detection](#section_2_2_3)
    * [Resize](Section_2_3)
* [Global Navigation](#chapter3)
    * [Occupancy Grid](#section_3_1)
    * [A* Algorithm](#section_3_2)
    
* [Kalman Filter](#chapter4)
    * [Prediction](#section_4_1)
    * [Update](#section_4_2)
   
* [Local Navigation](#chapter5)
    * [Local Avoidance](#section_5_1)
    * [Kidnapping](#section_5_2)
    
* [Overall Project](#chapter6)
    * [Overview](#section_6_1)
    
* [Conclusion](#chapter7)
* [Sources](#chapter8)

### Introduction <a class="anchor" id="chapter1"></a>



#### Framework  <a class="anchor" id="section_1_1"></a>

![frame](framework.jpeg)







#### Modules <a class="anchor" id="section_1_2"></a>

describe modules 

#### Environment <a class="anchor" id="section_1_3"></a>

![frame](Final_map.jpg)

### Computer Vision <a class="anchor" id="chapter2"></a>

#### Vision initialization and calibration <a class="anchor" id="section_2_1"></a>

#### Prepocessing <a class="anchor" id="section_2_2"></a>

#### Map initialization <a class="anchor" id="section_2_3"></a>

##### Obstacles Detection <a class="anchor" id="section_2_3_1"></a>

##### Goal Detection <a class="anchor" id="section_2_3_2"></a>

##### Thymio Detection <a class="anchor" id="section_2_3_3"></a>




### Global Navigation <a class="anchor" id="chapter3"></a>

#### Occupancy Grid<a class="anchor" id="section_3_1"></a>

#### A* Algorithm <a class="anchor" id="section_3_2"></a>


### Kalman Filter <a class="anchor" id="chapter4"></a>

#### Prediction<a class="anchor" id="section_4_1"></a>

#### Update <a class="anchor" id="section_4_2"></a>



### Local Navigation <a class="anchor" id="chapter5"></a>

#### Local Avoidance <a class="anchor" id="section_5_1"></a>

#### Kidnapping <a class="anchor" id="section_5_2"></a>



 

### Overall Project <a class="anchor" id="chapter6"></a>

#### Overview <a class="anchor" id="section_6_1"></a>
#### Main Function <a class="anchor" id="section_6_2"></a>

### Conclusion<a class="anchor" id="chapter7"></a>

    




### Sources <a class="anchor" id="chapter8"></a>

All sources used for the project from internet
Maybe we shouuld also talk about the session that we use from the course 


#### source 1 <a class="anchor" id="section_8_1"></a>

#### source 2 <a class="anchor" id="section_8_2"></a>